# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from jplephem.spk import SPK
import astropy.units as u
from astropy.time import Time
from astropy.utils.data import download_file

PLANET_EPHEMERIS_URL = "https://ssd.jpl.nasa.gov//ftp/eph/planets/bsp/de440.bsp"
ASTEROID_EPHEMERIS_URL = (
    "https://ssd.jpl.nasa.gov/ftp/eph/small_bodies/asteroids_de441/sb441-n16.bsp"
)

In [2]:
kernel = SPK.open(download_file(PLANET_EPHEMERIS_URL, cache=True))
dir(kernel)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'close',
 'comments',
 'daf',
 'open',
 'pairs',
 'segments']

In [1]:
from jorbit.ephemeris import Ephemeris

e = Ephemeris()

In [3]:
e.ssos

['mercury',
 'venus',
 'earth',
 'mars',
 'jupiter',
 'saturn',
 'uranus',
 'neptune',
 'pluto',
 'sun']

In [17]:
%timeit e.state(Time("2021-01-01T00:00:00"))[10]["x"].block_until_ready()

1.58 ms ± 19.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
from astroquery.jplhorizons import Horizons

obj = Horizons(
    id="3",
    epochs=Time("2021-01-01T00:00:00").tdb.jd,
    id_type="majorbody",
    location="@0",
)
horizons_results = obj.vectors(refplane="earth")
horizons_calc = jnp.array(
    [horizons_results["x"][0], horizons_results["y"][0], horizons_results["z"][0]]
)
horizons_results

targetname,datetime_jd,datetime_str,x,y,z,vx,vy,vz,lighttime,range,range_rate
---,d,---,AU,AU,AU,AU / d,AU / d,AU / d,d,AU,AU / d
str25,float64,str30,float64,float64,float64,float64,float64,float64,float64,float64,float64
Earth-Moon Barycenter (3),2459215.50080074,A.D. 2021-Jan-01 00:01:09.1839,-0.1857985333448159,0.8925393953089126,0.3870295356607404,-0.01720372762894869,-0.002940779545245006,-0.001274533654563454,0.005720211468932683,0.9904239136073262,7.914146143481989e-05


In [20]:
e = Ephemeris()
jorbit_calc = e.state(Time("2021-01-01T00:00:00"))
jorbit_calc = jorbit_calc[3]["x"]
jorbit_calc

Array([-0.18579853,  0.8925394 ,  0.38702954], dtype=float64)

In [21]:
jnp.linalg.norm(jorbit_calc - horizons_calc) * u.au.to(u.m)

Array(0.19863445, dtype=float64)

In [22]:
# 0.2 meters! what about jplephem?
jplephem_calc = kernel.segments[2].compute(
    Time("2021-01-01T00:00:00").tdb.jd
) * u.km.to(u.au)
jnp.linalg.norm(jplephem_calc - horizons_calc) * u.au.to(u.m)

Array(0.20586389, dtype=float64)

In [23]:
jnp.linalg.norm(jplephem_calc - jorbit_calc) * u.au.to(u.mm)

Array(7.22946136, dtype=float64)

In [24]:
# 7mm difference between jplephem and jorbit, 0.2m between each and Horizons

In [25]:
for k in kernel.segments:
    print(k.target)
    print(k.describe())

1
1549-12-31..2650-01-25  Type 2  Solar System Barycenter (0) -> Mercury Barycenter (1)
  frame=1 source=DE-0440LE-0440
2
1549-12-31..2650-01-25  Type 2  Solar System Barycenter (0) -> Venus Barycenter (2)
  frame=1 source=DE-0440LE-0440
3
1549-12-31..2650-01-25  Type 2  Solar System Barycenter (0) -> Earth Barycenter (3)
  frame=1 source=DE-0440LE-0440
4
1549-12-31..2650-01-25  Type 2  Solar System Barycenter (0) -> Mars Barycenter (4)
  frame=1 source=DE-0440LE-0440
5
1549-12-31..2650-01-25  Type 2  Solar System Barycenter (0) -> Jupiter Barycenter (5)
  frame=1 source=DE-0440LE-0440
6
1549-12-31..2650-01-25  Type 2  Solar System Barycenter (0) -> Saturn Barycenter (6)
  frame=1 source=DE-0440LE-0440
7
1549-12-31..2650-01-25  Type 2  Solar System Barycenter (0) -> Uranus Barycenter (7)
  frame=1 source=DE-0440LE-0440
8
1549-12-31..2650-01-25  Type 2  Solar System Barycenter (0) -> Neptune Barycenter (8)
  frame=1 source=DE-0440LE-0440
9
1549-12-31..2650-01-25  Type 2  Solar System Ba

In [34]:
kernel.segments[-3]._data[-1].shape

(13, 3, 100448)

In [40]:
jnp.log(6.311034342087888e-15), jnp.log(2.9591220828411951e-04)

(Array(-32.69647681, dtype=float64, weak_type=True),
 Array(-8.12544774, dtype=float64, weak_type=True))